In [1]:
import os
from os import walk
import numpy as np
from random import uniform
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img

from img_helper import rgb2hsv_set, illuminate_images, add_noise

# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

dataset_path = "ExDark/"
dataset_npy_path = "Dataset npys/"

In [2]:
# Image parameters
img_width = 100
img_height = img_width
img_channels = 3

# Load in all images from the specified folder
def create_datasets(img_folder, tt_split, vt_split):

    img_files=[]
    for (dir_path, dir_names, file_names) in walk(img_folder):
        for fn in file_names:
            if fn is not ".ipynb_checkpoints":
                img_files.append(os.path.join(dir_path,fn))
    
    img_array = np.ndarray(shape=(len(img_files), img_width, img_height, img_channels),
                         dtype=np.float32)

    i = 0
    for _file in img_files:
    	#img_path = img_folder + "/" + _file
    	img = load_img(_file, target_size=(img_width, img_height))
    	x = img_to_array(img)
    	img_array[i] = x
    	i += 1
    
    # Rescale the pixel values to range [0,1]
    img_array = img_array/np.max(img_array)

    # Split dataset into: 99% training and 1% test
    train,test = train_test_split(img_array, test_size=tt_split, random_state=13)

    # Split training set into: 80% training and 20% validation
    train,valid = train_test_split(train, test_size=vt_split, random_state=13)

    return train,test,valid

# Create image datasets
train_RGB_X,test_RGB_X,valid_RGB_X = create_datasets(dataset_path, 0.01, 0.2)

np.save(os.path.join(dataset_npy_path,"train_RGB_X.npy"),train_RGB_X)
np.save(os.path.join(dataset_npy_path,"test_RGB_X.npy"),test_RGB_X)
np.save(os.path.join(dataset_npy_path,"valid_RGB_X.npy"),valid_RGB_X)

In [3]:
# Modify datasets for networks
train_HSV_X = rgb2hsv_set(train_RGB_X)
valid_HSV_X = rgb2hsv_set(valid_RGB_X)

train_HSV_Y = illuminate_images(train_HSV_X)
valid_HSV_Y = illuminate_images(valid_HSV_X)

train_noisy_HSV_Y = add_noise(train_HSV_Y)
valid_noisy_HSV_Y = add_noise(valid_HSV_Y)

test_HSV_X = rgb2hsv_set(test_RGB_X)

np.save(os.path.join(dataset_npy_path,"train_HSV_X.npy"),train_HSV_X)
np.save(os.path.join(dataset_npy_path,"valid_HSV_X.npy"),valid_HSV_X)

np.save(os.path.join(dataset_npy_path,"train_HSV_Y.npy"),train_HSV_Y)
np.save(os.path.join(dataset_npy_path,"valid_HSV_Y.npy"),valid_HSV_Y)

np.save(os.path.join(dataset_npy_path,"train_noisy_HSV_Y.npy"),train_noisy_HSV_Y)
np.save(os.path.join(dataset_npy_path,"valid_noisy_HSV_Y.npy"),valid_noisy_HSV_Y)

np.save(os.path.join(dataset_npy_path,"test_HSV_X.npy"),test_HSV_X)